In [6]:
import pandas as pd
import numpy as np

def get_highly_correlated_pairs(df: pd.DataFrame, threshold: float = 0.95):
    """
    データフレーム中のカラム間で、相関係数が指定した閾値以上の組み合わせを一覧で取得する関数
    
    Parameters
    ----------
    df : pd.DataFrame
        対象のデータフレーム（数値データのみ）
    threshold : float, default=0.95
        相関係数の閾値
    
    Returns
    -------
    correlated_pairs : pd.DataFrame
        カラムペアとその相関係数をまとめたDataFrame
    """
    # 相関行列を計算（絶対値を取る）
    corr_matrix = df.corr().abs()

    # 上三角行列のみを対象にして、重複を除く
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )

    # 閾値を超えるペアを抽出
    correlated_pairs = (
        upper.stack()
        .reset_index()
        .rename(columns={"level_0": "Column1", "level_1": "Column2", 0: "Correlation"})
    )
    correlated_pairs = correlated_pairs[correlated_pairs["Correlation"] >= threshold]

    # 相関係数の高い順にソート
    correlated_pairs = correlated_pairs.sort_values(by="Correlation", ascending=False).reset_index(drop=True)

    return correlated_pairs


In [7]:
# サンプルデータ
data = {
    "A": [1, 2, 3, 4, 5],
    "B": [2, 4, 6, 8, 10],
    "C": [5, 4, 3, 2, 1],
    "D": [1, 1, 1, 1, 1]
}

df = pd.DataFrame(data)

# 関数呼び出し
result = get_highly_correlated_pairs(df, threshold=0.95)
print(result)



  Column1 Column2  Correlation
0       A       B          1.0
1       A       C          1.0
2       B       C          1.0


In [8]:
def get_highly_correlated_pairs(df: pd.DataFrame, threshold: float = 0.95):
    """
    Binary特徴量や数値データにおいて、
    相関係数が指定した閾値以上のカラムペアを取得する関数
    
    Parameters
    ----------
    df : pd.DataFrame
        データフレーム（0/1などのBinaryも可）
    threshold : float
        相関係数の閾値（例：0.95）
    
    Returns
    -------
    correlated_pairs : pd.DataFrame
        カラムペアと相関係数の一覧
    """
    # 定数カラムを除外
    df = df.loc[:, df.std() > 0]

    # 相関行列を計算（絶対値）
    corr_matrix = df.corr().abs()

    # 上三角のみ抽出
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # 閾値以上を抽出
    correlated_pairs = (
        upper.stack()
        .reset_index()
        .rename(columns={"level_0": "Column1", "level_1": "Column2", 0: "Correlation"})
    )
    correlated_pairs = correlated_pairs[correlated_pairs["Correlation"] >= threshold]
    correlated_pairs = correlated_pairs.sort_values(by="Correlation", ascending=False).reset_index(drop=True)

    return correlated_pairs


In [9]:
# 疑似MFPデータ
data = {
    "bit1": [1, 0, 1, 0, 1],
    "bit2": [1, 0, 1, 0, 1],  # 完全一致
    "bit3": [0, 1, 0, 1, 0],  # 反転関係
    "bit4": [1, 1, 1, 1, 1],  # 定数列
}

df = pd.DataFrame(data)

result = get_highly_correlated_pairs(df, threshold=0.9)
print(result)


  Column1 Column2  Correlation
0    bit1    bit2          1.0
1    bit1    bit3          1.0
2    bit2    bit3          1.0


In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import hamming

def get_highly_similar_pairs(df: pd.DataFrame, threshold: float = 0.9, metric: str = "jaccard"):
    """
    Binary特徴量（例: Morgan Fingerprint）の類似度に基づき、
    類似度が閾値以上のカラムペアを取得する関数
    
    Parameters
    ----------
    df : pd.DataFrame
        Binary (0/1) 特徴量のDataFrame
    threshold : float, default=0.9
        類似度の閾値
    metric : str, default="jaccard"
        類似度の種類
        - "jaccard"  : 共有1の割合
        - "cosine"   : コサイン類似度
        - "matching" : 全体一致度（1と0両方を考慮）
    
    Returns
    -------
    similar_pairs : pd.DataFrame
        カラムペアと類似度をまとめたDataFrame
    """
    # 定数列（全0または全1）は除外
    df = df.loc[:, df.std() > 0]
    cols = df.columns
    n = len(cols)
    results = []

    for i in range(n):
        for j in range(i + 1, n):
            x = df.iloc[:, i].values
            y = df.iloc[:, j].values

            if metric == "jaccard":
                sim = jaccard_score(x, y)
            elif metric == "cosine":
                sim = cosine_similarity(x.reshape(1, -1), y.reshape(1, -1))[0, 0]
            elif metric == "matching":
                sim = 1 - hamming(x, y)
            else:
                raise ValueError("metric must be one of ['jaccard', 'cosine', 'matching']")

            if sim >= threshold:
                results.append((cols[i], cols[j], sim))

    similar_pairs = pd.DataFrame(results, columns=["Column1", "Column2", "Similarity"])
    similar_pairs = similar_pairs.sort_values(by="Similarity", ascending=False).reset_index(drop=True)
    return similar_pairs


In [11]:
# 疑似フィンガープリントデータ
data = {
    "bit1": [1, 0, 1, 0, 1],
    "bit2": [1, 0, 1, 0, 1],  # 完全一致
    "bit3": [0, 1, 0, 1, 0],  # bit1と反転
    "bit4": [1, 1, 1, 1, 1],  # 定数列（除外される）
}
df = pd.DataFrame(data)

# Jaccard類似度
result_jaccard = get_highly_similar_pairs(df, threshold=0.9, metric="jaccard")
print("Jaccard:\n", result_jaccard, "\n")

# Cosine類似度
result_cosine = get_highly_similar_pairs(df, threshold=0.9, metric="cosine")
print("Cosine:\n", result_cosine, "\n")

# Matching係数
result_matching = get_highly_similar_pairs(df, threshold=0.9, metric="matching")
print("Matching:\n", result_matching)


Jaccard:
   Column1 Column2  Similarity
0    bit1    bit2         1.0 

Cosine:
   Column1 Column2  Similarity
0    bit1    bit2         1.0 

Matching:
   Column1 Column2  Similarity
0    bit1    bit2         1.0
